In [1]:
import pandas as pd
import numpy as np

In [ ]:
import indicators
import myutiles
import sys

In [ ]:
def MACD(data, short_window, long_window):
    try:
        
        if (len(data) < long_window):
            print(sys._getframe().f_code.co_name,'Insufficient data')
            return None
        
        if (short_window < 1):
            print(sys._getframe().f_code.co_name,'Invalid parameter value for short_window')
            return None
        
        if (long_window <= short_window):
            print(sys._getframe().f_code.co_name,'Invalid parameter value for long_window')
            return None
        
        data = pd.DataFrame(data[-long_window:])
        
#        data['Short_MA'] = data['Close'].rolling(window = short_window, center = False).mean()
        data['Short_MA'] = data['Close'].ewm(span = short_window, adjust = False).mean()
        
#        data['Long_MA'] = data['Close'].rolling(window = long_window, center = False).mean()
        data['Long_MA'] = data['Close'].ewm(span = long_window, adjust = False).mean()
        
        data['MACD'] = data['Short_MA'] - data['Long_MA']
        
        MACD = float(data['MACD'].iloc[-1])
        
        return MACD
        
    except Exception as ex:
        print(sys._getframe().f_code.co_name, 'exception :', ex)
        return None



In [ ]:
def generate_signal_MACD(symbol_name, ltp, short_window, long_window, long_entry, short_entry, data_path ):
    try:
        
        symbol_name = symbol_name.strip()
        
        if type(ltp) is str:
            ltp = float(ltp.strip())
        
        if type(short_window) is str:
            short_window = int(short_window.strip())
       
        if type(long_window) is str:
            long_window = int(long_window.strip())
        
        if type(long_entry) is str:
            long_entry = float(long_entry.strip())
        
        if type(short_entry) is str:
            short_entry = float(short_entry.strip())
        
        if (symbol_name == '' or ltp <= 0 or short_window < 1 or long_window <= short_window or long_entry < 1 or short_entry <= 0 or short_entry > 1):
            print(sys._getframe().f_code.co_name,'Invalid parameter values')
            return None
        
        data = myutils.read_dataframe(data_path + '/' + symbol_name + '.csv')
        
        if (data.empty):
            print(sys._getframe().f_code.co_name,'Data not found')
            return None
        
        # specific to your data set, modify accordingly
        data = data['Close']
        
        data = data.dropna()
        
        #data = data[-period:]
        
        current_macd = indicators.MACD(data, short_window, long_window)
        
        if (current_macd == None):
            return None
        
        print('MACD ' + str(current_macd))
        
        data['Long_MA'] = data['Close'][-long_window:].ewm(span = long_window, adjust = False).mean()
        
        LMA = data['Long_MA'].iloc[-1]
        
        if (current_macd > LMA * long_entry):
            signal = 1
        elif (current_macd < -LMA * short_entry):
            signal = -1
        else:
            signal = 0
        
        return signal
        
    except Exception as ex:
        print(sys._getframe().f_code.co_name, 'exception :', ex)
        return None